In [1]:
%reload_ext autoreload
%autoreload 2
from kaggle import Kaggle

### EDA Steps

In [2]:
# Create Kaggle Object
kag = Kaggle(data_path='data/', metric='auc')
# Read Train Data
kag.read_train_data(train_name='train.csv', sep='\t', target_colname=None, id_colname=None)
# Perform EDA on Train Data
kag.df_train.head(1)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,336,337,338,339,340,341,342,343,344,345
0,0,1,1,0,0,0,0,0,0.090909,0,...,0.221395,0,1,0,0,0.222222,1,1,1,1


In [3]:
# Read Test Data if needed
kag.read_train_data(train_name='test.csv', sep='\t', target_colname=None, id_colname=None)

### Validation Steps

In [4]:
# Create Kaggle Object
kag = Kaggle(data_path='data/', metric='auc')
# Read Train Data with Target and ID columns already known
kag.read_train_data(train_name='train.csv', sep='\t', target_colname='0', id_colname='Unnamed: 0')

In [5]:
# Create validation splits and save them
kag.create_validation_split(n_folds=5, stratified=False)

In [6]:
# Perform FE which does not depend on Train/Test split
kag.general_feature_engineering(train_only=True)

In [7]:
# Run Single (e.g. XGBoost) model. With additional FE inside folds.
params = {'booster': 'gbtree',
            'eta': .1,
            'colsample_bytree': 0.8,
            'subsample': 0.8,
            'seed': 123,
            'nthread': 3,
            'max_depth': 6,
            'min_child_weight': 1,
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'silent': 1
        }
score_mean, score_std, overall_score = kag.run_single_model_validation(model_name='xgboost',
                                                                       params=params, 
                                                                       oof_preds_path='')

************************** FOLD 1 **************************
[0]	train-auc:0.72341+0.00464833	test-auc:0.690028+0.00933701
[50]	train-auc:0.876934+0.00454843	test-auc:0.725266+0.00696124


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


Best Iteration: 60
0.7458752423515378
************************** FOLD 2 **************************
[0]	train-auc:0.726342+0.00366855	test-auc:0.688941+0.00140703
[50]	train-auc:0.879546+0.0029343	test-auc:0.729213+0.0031834
[100]	train-auc:0.9343+0.000927095	test-auc:0.729638+0.00225026
Best Iteration: 87
0.7332484322367749
************************** FOLD 3 **************************
[0]	train-auc:0.717133+0.0119537	test-auc:0.681674+0.00923283
[50]	train-auc:0.876646+0.00367825	test-auc:0.730528+0.00483306
[100]	train-auc:0.934798+0.00268595	test-auc:0.730951+0.00356462
Best Iteration: 127
0.7380085132259045
************************** FOLD 4 **************************
[0]	train-auc:0.725549+0.00238688	test-auc:0.688564+0.00067813
[50]	train-auc:0.877899+0.00399425	test-auc:0.730316+0.00482027
Best Iteration: 63
0.7344492421612866
************************** FOLD 5 **************************
[0]	train-auc:0.724051+0.000509871	test-auc:0.685681+0.00681387
[50]	train-auc:0.879152+0.001831

In [8]:
# # Find best params with a higher LR (for a speed). Takes loong time
# best_params = kag.find_optimal_params(model_name='xgboost')

# # Rerun model with best params and LR corrected
# best_params['eta'] = 0.1
# kag.run_single_model_validation(model_name='xgboost', params=best_params, oof_preds_path='')

### 2nd Level Modeling

In [9]:
# Create Kaggle Object
kag = Kaggle(data_path='data/', metric='auc')
# Read Train Data with Target and ID columns already known
kag.read_train_data(train_name='train.csv', sep='\t', target_colname='0', id_colname='Unnamed: 0')
# Perform FE which does not depend on Train/Test split
kag.general_feature_engineering(train_only=True)

# Run Single Models and Save OOF predictions
xgb_params = {'booster': 'gbtree',
            'eta': .1,
            'colsample_bytree': 0.8,
            'subsample': 0.8,
            'seed': 123,
            'nthread': 3,
            'max_depth': 6,
            'min_child_weight': 1,
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'silent': 1
        }
kag.run_single_model_validation(model_name='xgboost',
                                params=xgb_params,
                                oof_preds_path='oof_preds_level_1/xgboost_0.csv')

lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 8,
        'learning_rate': 0.1,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
        }  
kag.run_single_model_validation(model_name='lightgbm',
                                params=lgb_params,
                                oof_preds_path='oof_preds_level_1/lightgbm_0.csv')

************************** FOLD 1 **************************
[0]	train-auc:0.72341+0.00464833	test-auc:0.690028+0.00933701
[50]	train-auc:0.876934+0.00454843	test-auc:0.725266+0.00696124


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


Best Iteration: 60
0.7458752423515378
************************** FOLD 2 **************************
[0]	train-auc:0.726342+0.00366855	test-auc:0.688941+0.00140703
[50]	train-auc:0.879546+0.0029343	test-auc:0.729213+0.0031834
[100]	train-auc:0.9343+0.000927095	test-auc:0.729638+0.00225026
Best Iteration: 87
0.7332484322367749
************************** FOLD 3 **************************
[0]	train-auc:0.717133+0.0119537	test-auc:0.681674+0.00923283
[50]	train-auc:0.876646+0.00367825	test-auc:0.730528+0.00483306
[100]	train-auc:0.934798+0.00268595	test-auc:0.730951+0.00356462
Best Iteration: 127
0.7380085132259045
************************** FOLD 4 **************************
[0]	train-auc:0.725549+0.00238688	test-auc:0.688564+0.00067813
[50]	train-auc:0.877899+0.00399425	test-auc:0.730316+0.00482027
Best Iteration: 63
0.7344492421612866
************************** FOLD 5 **************************
[0]	train-auc:0.724051+0.000509871	test-auc:0.685681+0.00681387
[50]	train-auc:0.879152+0.001831

(0.73693, 0.00397, 0.73297)

In [10]:
# Run Second Level Model
kag.run_stacked_model_validation(model_name='logistic_regression',
                                 params=None,
                                 prev_level_fold='oof_preds_level_1/',
                                 oof_preds_path='')

************************** FOLD 1 **************************
0.7487024408350564
************************** FOLD 2 **************************
0.7368546789896248
************************** FOLD 3 **************************
0.7420741486774094
************************** FOLD 4 **************************
0.7368555362721276
************************** FOLD 5 **************************
0.742443048641812
auc mean: 0.74139, auc std: 0.00439, auc overall: 0.737
ALL FOLDS: [0.7487, 0.73685, 0.74207, 0.73686, 0.74244]


(0.74139, 0.00439, 0.737)

### Stacked Model Test Prediction

In [24]:
kag = Kaggle(data_path='data/', metric='auc')
kag.read_train_data(train_name='train.csv', sep='\t', target_colname='0', id_colname='Unnamed: 0')
kag.read_test_data(test_name='train.csv')
kag.general_feature_engineering(train_only=False)

xgb_params = {'booster': 'gbtree',
            'eta': .1,
            'colsample_bytree': 0.8,
            'subsample': 0.8,
            'seed': 123,
            'nthread': 3,
            'max_depth': 6,
            'min_child_weight': 1,
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'silent': 1
        }

test_preds_1 = kag.get_single_model_test_prediction(model_name='xgboost',
                                                    params=xgb_params,
                                                    preds_path='test_preds_level_1/xgboost_0.csv')

lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 8,
        'learning_rate': 0.1,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
        }  
test_preds_2 = kag.get_single_model_test_prediction(model_name='lightgbm',
                                                    params=lgb_params,
                                                    preds_path='test_preds_level_1/lightgbm_0.csv')

# Run Second Level Model
test_preds = kag.get_stacked_model_test_prediction(model_name='logistic_regression',
                                               params=None,
                                               prev_level_test_fold='test_preds_level_1/',
                                               preds_path='test_preds_level_2/stack_pred.csv')
test_preds.head()

[0]	train-auc:0.722864+0.00421065	test-auc:0.694596+0.00834651
[50]	train-auc:0.860283+0.00145325	test-auc:0.734999+0.00547926
[100]	train-auc:0.916391+0.0017179	test-auc:0.735714+0.00511333


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


Best Iteration: 144
[50]	cv_agg's auc: 0.730732 + 0.0023251
[100]	cv_agg's auc: 0.734091 + 0.00202989
[150]	cv_agg's auc: 0.735014 + 0.00177441
[200]	cv_agg's auc: 0.734382 + 0.00180827
Best Iteration: 225


,index,stack_pred
0,0,0.761878
1,1,0.147439
2,2,0.129632
3,3,0.127195
4,4,0.682124
